In [4]:
#pip install smithery langchain-mcp-tools

In [11]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
import os 
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
SMITHERY_API_KEY = os.getenv("SMITHERY_API_KEY")

python_repl = PythonREPL()

print(python_repl.run("print('hello world')"))

# install smithery, langchain_mcp_tools
import smithery
import mcp
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# MCP To LangChain Tools Conversion Utility, install langchain_mcp_tools
# https://pypi.org/project/langchain-mcp-tools/ 

# login to Smithery: https://smithery.ai/
# create a API Key
llm = ChatOpenAI(model="gpt-4o")

hello world



In [6]:
# install smithery, langchain_mcp_tools
import smithery
import mcp
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# MCP To LangChain Tools Conversion Utility, install langchain_mcp_tools
# https://pypi.org/project/langchain-mcp-tools/ 

# login to Smithery: https://smithery.ai/
# create a API Key
llm = ChatOpenAI(model="gpt-4o")

In [32]:
mcp_servers = {
    "filesystem": {
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "."]
    },

}

tools, cleanup = await convert_mcp_to_langchain_tools(
    mcp_servers
)

[INFO] MCP server "filesystem": initializing with: {'command': 'npx', 'args': ['-y', '@modelcontextprotocol/server-filesystem', '.']}
[INFO] MCP server "filesystem": connected
[INFO] MCP server "filesystem": 11 tool(s) available:
[INFO] - read_file
[INFO] - read_multiple_files
[INFO] - write_file
[INFO] - edit_file
[INFO] - create_directory
[INFO] - list_directory
[INFO] - directory_tree
[INFO] - move_file
[INFO] - search_files
[INFO] - get_file_info
[INFO] - list_allowed_directories
[INFO] MCP servers initialized: 11 tool(s) available in total


In [42]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_core.messages import BaseMessage, HumanMessage

class MCPAgent:
    def __init__(self, llm):
        self.llm = llm
        self.mcp_servers = {
            "filesystem": {
                "command": "npx",
                "args": ["@modelcontextprotocol/server-filesystem", os.getcwd()]
            }
        }
        self.tools = None
        self.agent = None
        self.cleanup_func = None
        self.config = {"configurable": {"thread_id": "42"}}

    async def initialize(self):
        self.tools, self.cleanup_func = await convert_mcp_to_langchain_tools(self.mcp_servers)
        if self.tools:
            self.agent = create_react_agent(self.llm, self.tools)
        else:
            print("Warning: No tools were loaded from MCP servers.")

    async def cleanup(self):
        if self.cleanup_func:
            await self.cleanup_func()
            print("MCP server sessions closed.")

    async def astream_events(self, query, config, chat_history=None, return_direct=False):
        if not self.agent:
            print("Agent not initialized.")
            return
        inputs = {"messages": [HumanMessage(content=query)]}
        async for chunk in self.agent.astream_events(inputs, config, stream_mode="values"):
            yield chunk

#async def main():
llm = ChatOpenAI(model="gpt-4o-mini")
# Create the MCPAgent instance
mcp_agent = MCPAgent(llm)

# Initialize the agent and load tools
await mcp_agent.initialize()

message = "List all files in the current directory"

message 
if mcp_agent.agent:
    event_stream = mcp_agent.astream_events(
        query=message,
        config=mcp_agent.config,
    )
    content = ""
    async for event in event_stream:
        #if isinstance(chunk, dict):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            content += chunk.content
            #print(repr(content))
            print(content)
            #message = chunk["messages"][-1]
            #if hasattr(message, 'content'):
            #    print(message.content)

# Cleanup the MCP server sessions
await mcp_agent.cleanup()

[INFO] MCP server "filesystem": initializing with: {'command': 'npx', 'args': ['@modelcontextprotocol/server-filesystem', '/home/teodorrk/projects/Bolt-x-OpenBridge/fastagent/teodor/langchain_mcp/notebooks']}
[INFO] MCP server "filesystem": connected
[INFO] MCP server "filesystem": 11 tool(s) available:
[INFO] - read_file
[INFO] - read_multiple_files
[INFO] - write_file
[INFO] - edit_file
[INFO] - create_directory
[INFO] - list_directory
[INFO] - directory_tree
[INFO] - move_file
[INFO] - search_files
[INFO] - get_file_info
[INFO] - list_allowed_directories
[INFO] MCP servers initialized: 11 tool(s) available in total
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] MCP tool "filesystem"/"list_allowed_directories" received input: {'path': '.'}
[INFO] MCP tool "filesystem"/"list_allowed_directories" received result (size: 17)


[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



The
The current
The current directory
The current directory contains
The current directory contains the
The current directory contains the following
The current directory contains the following file
The current directory contains the following file:


The current directory contains the following file:

-
The current directory contains the following file:

- **
The current directory contains the following file:

- **test
The current directory contains the following file:

- **test.ip


[INFO] MCP server "filesystem": session closed


The current directory contains the following file:

- **test.ipyn
The current directory contains the following file:

- **test.ipynb
The current directory contains the following file:

- **test.ipynb**
The current directory contains the following file:

- **test.ipynb**
MCP server sessions closed.


In [7]:
# Do not forget to provide your api key here
url1 = smithery.create_smithery_url("wss://server.smithery.ai/@swaroopkasaraneni/mcp-servers/ws", {}) + SMITHERY_API_KEY

url2 = smithery.create_smithery_url("wss://server.smithery.ai/@mcp-examples/weather/ws", {}) + SMITHERY_API_KEY

mcp_servers = {
    "server1": {"url": url1},
    "server2": {"url": url2},
}

tools, cleanup = await convert_mcp_to_langchain_tools(
    mcp_servers
)

print("Tools available:")
print(tools[0].get_name(), tools[1].get_name(), tools[2].get_name(), tools[3].get_name())

# Tools available:
# add multiply get-alerts get-forecast

# creating an agent

[INFO] MCP server "server1": initializing with: {'url': 'wss://server.smithery.ai/@swaroopkasaraneni/mcp-servers/ws?config=e30%3D&api_key=8c280430-cd8c-4708-ae5d-b87d0ba3345'}
[INFO] MCP server "server2": initializing with: {'url': 'wss://server.smithery.ai/@mcp-examples/weather/ws?config=e30%3D&api_key=8c280430-cd8c-4708-ae5d-b87d0ba3345'}
[INFO] MCP server "server1": connected
[INFO] MCP server "server1": 2 tool(s) available:
[INFO] - add
[INFO] - multiply
[INFO] MCP server "server2": connected
[INFO] MCP server "server2": 2 tool(s) available:
[INFO] - get-alerts
[INFO] - get-forecast
[INFO] MCP servers initialized: 4 tool(s) available in total


Tools available:
add multiply get-alerts get-forecast


In [10]:
import smithery
import mcp
from mcp.client.websocket import websocket_client

# Create Smithery URL with server endpoint
file_url = smithery.create_smithery_url("wss://server.smithery.ai/@bunasQ/fs/ws", {}) + SMITHERY_API_KEY

context_url = url = smithery.create_smithery_url("wss://server.smithery.ai/@bsmi021/mcp-file-context-server/ws", {}) + SMITHERY_API_KEY


mcp_servers = {
    "file_server": {"url": file_url},
    "context_server": {"url": context_url},
}

tools, cleanup = await convert_mcp_to_langchain_tools(
    mcp_servers
)
print(f"Available tools: {', '.join([t.name for t in tools])}")

file_agent = create_react_agent(
    llm,
    tools
)
message = "Can you read the contents of the file 'requirements.txt' in this path: '/home/teodorrk/projects/openbridge-mvp/teodor/mcp-agent-langgraph/requirements.txt'?"
agent_response = await file_agent.ainvoke({"messages": message})
agent_response

[INFO] MCP server "file_server": initializing with: {'url': 'wss://server.smithery.ai/@bunasQ/fs/ws?config=e30%3D&api_key=8c280430-cd8c-4708-ae5d-b87d0ba3345'}
[INFO] MCP server "context_server": initializing with: {'url': 'wss://server.smithery.ai/@bsmi021/mcp-file-context-server/ws?config=e30%3D&api_key=8c280430-cd8c-4708-ae5d-b87d0ba3345'}
[INFO] MCP server "file_server": connected
[INFO] MCP server "file_server": 1 tool(s) available:
[INFO] - read-file-21
[INFO] MCP server "context_server": connected
[INFO] MCP server "context_server": 5 tool(s) available:
[INFO] - read_context
[INFO] - get_chunk_count
[INFO] - set_profile
[INFO] - get_profile_context
[INFO] - generate_outline
[INFO] MCP servers initialized: 6 tool(s) available in total


Available tools: read-file-21, read_context, get_chunk_count, set_profile, get_profile_context, generate_outline


[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] MCP tool "context_server"/"generate_outline" received input: {'path': '/home/teodorrk/projects/openbridge-mvp/teodor/mcp-agent-langgraph/requirements.txt'}
[WARNING] MCP tool "context_server"/"generate_outline" caused error:  Tool execution failed: [TextContent(type='text', text="File operation error: Failed to read file: ENOENT: no such file or directory, stat '/home/teodorrk/projects/openbridge-mvp/teodor/mcp-agent-langgraph/requirements.txt' (UNKNOWN_ERROR)", annotations=None)]
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
[INFO] Retrying request to /chat/completions in 0.243000 seconds
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
[INFO] Retrying request to /chat/completions in 0.243000 seconds
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-535DL63px2GlJwXoOpldBPle on tokens per min (TPM): Limit 30000, Used 30000, Requested 122. Please try again in 243ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [8]:
url = smithery.create_smithery_url("wss://server.smithery.ai/@bsmi021/mcp-file-context-server/ws", {}) + "&api_key=your-smithery-api-key"


## Github Tool

In [ ]:
# Create Smithery URL with server endpoint
git_url = smithery.create_smithery_url("wss://server.smithery.ai/@smithery-ai/github/ws", {
  "githubPersonalAccessToken": GITHUB_TOKEN
}) + SMITHERY_API_KEY

mcp_servers = {
    "git_server": {"url": git_url},
}

tools, cleanup = await convert_mcp_to_langchain_tools(
    mcp_servers
)

print(f"Available tools: {', '.join([t.name for t in tools])}")
            


[INFO] MCP server "git_server": initializing with: {'url': 'wss://server.smithery.ai/@smithery-ai/github/ws?config=e30%3D8c280430-cd8c-4708-ae5d-b87d0ba3345'}


CancelledError: 

In [18]:
import smithery
import mcp
from mcp.client.websocket import websocket_client

# Create Smithery URL with server endpoint
url = smithery.create_smithery_url("wss://server.smithery.ai/@smithery-ai/github/ws", {
  "githubPersonalAccessToken": GITHUB_TOKEN
}) + SMITHERY_API_KEY

async def github_operations():
    # Connect to the server using websocket client
    async with websocket_client(url) as streams:
        async with mcp.ClientSession(*streams) as session:
            # List available tools (optional, to see what's available)
            tools_result = await session.list_tools()
            print(f"Available tools: {', '.join([t.name for t in tools_result])}")
            
            # # Example: Fetch repository info (adjust tool_name and params)
            # repo_info = await session.call_tool(
            #     "get_repository",  # Hypothetical tool name (check actual available tools)
            #     {
            #         "owner": "TeodorRusKvi",
            #         "public_repo": "Text-Analysis",
            #     }
            # )
            # print("Repository info:", repo_info)

            # Example: Create an issue (adjust tool_name and params)
            # new_issue = await session.call_tool(
            #     "create_issue",
            #     {
            #         "owner": "your-github-username",
            #         "repo": "your-repo-name",
            #         "title": "Bug: Something is broken",
            #         "body": "Detailed description of the issue..."
            #     }
            # )
            # print("Created issue:", new_issue)
await github_operations() 

CancelledError: 

In [5]:
agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
agent_response 

[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] MCP tool "server1"/"multiply" received input: {'a': 3, 'b': 5}
[INFO] MCP tool "server1"/"multiply" received result (size: 1)
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] MCP tool "server1"/"multiply" received input: {'a': 8, 'b': 12}
[INFO] MCP tool "server1"/"multiply" received result (size: 2)
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='757783ee-0300-4171-b05b-7242857561bd'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JisHTAVIcVggZjN7M3IwnjhI', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 149, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7faba9ef5', 'id': 'chatcmpl-BLktu6jmSEFyqrOtk8DN1a6gJ88z6', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5ba96186-719c-4450-beab-68dea005c8fe-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_JisHTAVIcVggZjN7M3IwnjhI', 'type': 'tool_call'}], usage_meta

In [15]:
agent_response = await agent.ainvoke({"messages": "How is the weather in st. louis, MO?"})
agent_response

[04/13/25 08:02:52] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=412224;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=505021;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     MCP tool "server2"/"get-forecast" received input:           ]8;id=956214;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/langchain_mcp_tools/langchain_mcp_tools.py\langchain_mcp_tools.py]8;;\:]8;id=279227;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/langchain_mcp_tools/langchain_mcp_tools.py#234\234]8;;\
                             {'latitude': 38.627003, 'longitude': -90.199402}                                      

[04/13/25 08:02:53] INFO     MCP tool "server2"/"get-forecast" received result (size:    ]8;id=37982;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/langchain_mcp_tools/langchain_mcp_tools.py\langchain_mcp_tools.py]8;;\:]8;id=659100;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/langchain_mcp_tools/langchain_mcp_tools.py#270\270]8;;\
                             1149)                                                                                 

[04/13/25 08:02:57] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=869680;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=594180;file:///home/teodorrk/projects/openbridge-mvp/.myvenv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

{'messages': [HumanMessage(content='How is the weather in st. louis, MO?', additional_kwargs={}, response_metadata={}, id='7fb8a719-4994-4c02-9eb2-5f0291b7309d'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AT13lsISd4Pg7tX9cGI6EWk5', 'function': {'arguments': '{"latitude":38.627003,"longitude":-90.199402}', 'name': 'get-forecast'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 148, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_b7faba9ef5', 'id': 'chatcmpl-BLkiZ73kwxLXYhgvyBwEsNLfSQpe9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2f344dc7-9daa-4180-b1ae-13174e39d5c3-0', tool_calls=[{'name': 'get-forecast', 'args': {'latitude': 38.627003, 